# Описание моделей

## semi-log

Зависимая переменная преобразуется логарифмически; это единственное различие между аддитивными моделями и полулогарифмическими моделями.



## log-log

В логарифмических моделях независимые переменные также подвергаются логарифмическому преобразованию в дополнение к целевой переменной.

# Импорт библиотек

In [1]:
!pip install pystan

In [2]:
import pandas as pd
import numpy as np
import pystan

# Загрузка и обработка данных

In [3]:
df = pd.read_csv('datasets/data.csv',delimiter = ';')
seas = pd.read_csv('datasets/seas.csv')

In [4]:
seas_cols = [col for col in seas.columns if 'seas_' in col]

In [5]:
df = pd.concat([df,seas[seas_cols]],axis = 1)

In [6]:
df[pd.get_dummies(df.holiday, prefix='holiday',drop_first = True).columns] =\
pd.get_dummies(df.holiday, prefix='holiday',drop_first = True)

In [7]:
df = df.drop(columns = 'holiday')

In [8]:
df = df[~df.index.isin(df[df['macro_gas_dpg'].str.contains('Feb')].index)]
df = df[~df.index.isin(df[df['costs_auddig'].str.contains('Jan')].index)]

In [9]:
week_start = df['week_start']

df = df.drop(columns = 'week_start').astype(float)

df['week_start'] = week_start

In [10]:
imp_cols=[col for col in df.columns if 'impressions_' in col]
costs_cols=[col for col in df.columns if 'costs_' in col]
macro_cols = [col for col in df.columns if 'macro_' in col]
dis_cols = [col for col in df.columns if 'discount_' in col]
stores_cols = [col for col in df.columns if 'stores_count' in col]
sales = [col for col in df.columns if 'sales' in col]
holiday = [col for col in df.columns if 'holiday_' in col]
base_vars = macro_cols+dis_cols+stores_cols+holiday+seas_cols

# Создание модели lin-reg + semi-log reg

## Создание функций

In [11]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [12]:
def apply_mean_center(x):
    mu = np.mean(x)
    xm = x/mu
    return xm, mu

Тут мы делаем функцию для линейного преобразования. Это является некой нормализацией данных для линейной регрессии 

In [13]:
def mean_center_trandform(df, cols):
    df_new = pd.DataFrame()
    sc = {}
    for col in cols:
        x = df[col].values
        df_new[col], mu = apply_mean_center(x)
        sc[col] = mu
    return df_new, sc

Тут мы производим логарифмическое преобразование

In [14]:
def mean_log1p_trandform(df, cols):
    df_new = pd.DataFrame()
    sc = {}
    for col in cols:
        x = df[col].values
        xm, mu = apply_mean_center(x)
        sc[col] = mu
        df_new[col] = np.log1p(xm)
    return df_new, sc

## Control model

На данном этапе мы обучаем простую линейную регрессию, к которой применяем mean_center_trandform преобразование для нормализации признаков

### Создание признаков

In [15]:
df_ctrl, sc_ctrl = mean_center_trandform(df, sales + macro_cols + dis_cols + stores_cols)

df_ctrl = df_ctrl.merge(df[holiday+seas_cols],how = 'left',on = df.index)
#df_ctrl = df_ctrl.join(df[holiday+seas_cols])

In [16]:
df_ctrl.head(5)

,key_0,sales,macro_ics_all,macro_gas_dpg,discount_edw,discount_pdm,stores_count,holiday_Christmas Day/Christmas Eve/Day after Christmas,holiday_Christmas Day/Day after Christmas,holiday_Christmas Day/Day after Christmas/NYE,...,seas_prd_12,seas_week_40,seas_week_41,seas_week_42,seas_week_43,seas_week_44,seas_week_45,seas_week_46,seas_week_47,seas_week_48
0,0,0.663858,0.878350,1.396277,0.0,1.088653,1.087402,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.725988,0.878350,1.391228,0.0,1.067596,1.087619,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.645612,0.878350,1.378411,0.0,1.020033,1.088486,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.632448,0.878350,1.371808,0.0,1.055949,1.088486,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.794133,0.900708,1.373362,0.0,1.064874,1.088486,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
pos_vars = [col for col in base_vars if col not in seas_cols]
X1 = df_ctrl[pos_vars].values

In [18]:
pn_vars = seas_cols
X2 = df_ctrl[pn_vars].values

### Обучение модели

Тут мы как бы обьясняем будущей модели что за что ответственно и передаем ей наши переменные X1,X2. Коэффициенты модель сама генерит и подбирает из распределений, судя по всему

In [19]:
ctrl_data = {
    'N': len(df_ctrl),
    'K1': len(pos_vars), 
    'K2': len(pn_vars), 
    'X1': X1,
    'X2': X2, 
    'y': df_ctrl['sales'].values,
    'max_intercept': min(df_ctrl['sales'])
}

In [20]:
ctrl_code1 = '''
data {
  int N; // number of observations
  int K1; // number of positive predictors
  int K2; // number of positive/negative predictors
  real max_intercept; // restrict the intercept to be less than the minimum y
  matrix[N, K1] X1;
  matrix[N, K2] X2;
  vector[N] y; 
}

parameters {
  vector<lower=0>[K1] beta1; // regression coefficients for X1 (positive)
  vector[K2] beta2; // regression coefficients for X2
  real<lower=0, upper=max_intercept> alpha; // intercept
  real<lower=0> noise_var; // residual variance
}

model {
  // Define the priors
  beta1 ~ normal(0, 1); 
  beta2 ~ normal(0, 1); 
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  // The likelihood
  y ~ normal(X1*beta1 + X2*beta2 + alpha, sqrt(noise_var));
}
'''

In [21]:
sm1 = pystan.StanModel(model_code=ctrl_code1, verbose=True)
fit1 = sm1.sampling(data=ctrl_data, iter=2000, chains=4)
fit1_result = fit1.extract()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_548939bc33801f8115bc26206558c913 NOW.
INFO:pystan:OS: linux, Python: 3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)], Cython 0.28.5


Compiling /tmp/pystan_aeo5wwfj/stanfit4anon_model_548939bc33801f8115bc26206558c913_3150023247614078872.pyx because it changed.
[1/1] Cythonizing /tmp/pystan_aeo5wwfj/stanfit4anon_model_548939bc33801f8115bc26206558c913_3150023247614078872.pyx
building 'stanfit4anon_model_548939bc33801f8115bc26206558c913_3150023247614078872' extension
creating /tmp/pystan_aeo5wwfj/tmp
creating /tmp/pystan_aeo5wwfj/tmp/pystan_aeo5wwfj
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -m64 -fPIC -m64 -fPIC -fPIC -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -I/tmp/pystan_aeo5wwfj -I/opt/conda/lib/python3.6/site-packages/pystan -I/opt/conda/lib/python3.6/site-packages/pystan/stan/src -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3 -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0 -I/opt/conda/lib/python3.6/site-packages/pys

### Тестирование модели

In [22]:
def extract_ctrl_model(fit_result, pos_vars=pos_vars, pn_vars=pn_vars, 
                       extract_param_list=False):
    ctrl_model = {}
    ctrl_model['pos_vars'] = pos_vars
    ctrl_model['pn_vars'] = pn_vars
    ctrl_model['beta1'] = fit_result['beta1'].mean(axis=0).tolist()
    ctrl_model['beta2'] = fit_result['beta2'].mean(axis=0).tolist()
    ctrl_model['alpha'] = fit_result['alpha'].mean()
    if extract_param_list:
        ctrl_model['beta1_list'] = fit_result['beta1'].tolist()
        ctrl_model['beta2_list'] = fit_result['beta2'].tolist()
        ctrl_model['alpha_list'] = fit_result['alpha'].tolist()
    return ctrl_model

In [23]:
def ctrl_model_predict(ctrl_model, df):
    pos_vars, pn_vars = ctrl_model['pos_vars'], ctrl_model['pn_vars'] 
    X1, X2 = df[pos_vars], df[pn_vars]
    beta1, beta2 = np.array(ctrl_model['beta1']), np.array(ctrl_model['beta2'])
    alpha = ctrl_model['alpha']
    y_pred = np.dot(X1, beta1) + np.dot(X2, beta2) + alpha
    return y_pred

In [24]:
base_sales_model = extract_ctrl_model(fit1_result, pos_vars=pos_vars, pn_vars=pn_vars)
base_sales = ctrl_model_predict(base_sales_model, df_ctrl)
df['base_sales'] = base_sales*sc_ctrl['sales']
# evaluate control model
print('mape: ', mean_absolute_percentage_error(df['sales'], df['base_sales']))

mape:  25.60843824


## Marketing Mix Model

В данном случае у нас используется semi-logarithmic model, поскольку мы преобразуем зависимую переменную логарифмически, как было указано выше (преобразуем sales и base sales)

In [25]:
df_mmm, sc_mmm = mean_log1p_trandform(df, ['sales', 'base_sales'])

mu_mdip = df[imp_cols].apply(np.mean, axis=0).values
max_lag = 8
num_media = len(imp_cols)
# padding zero * (max_lag-1) rows
X_media = np.concatenate((np.zeros((max_lag-1, num_media)), df[imp_cols].values), axis=0)
X_ctrl = df_mmm['base_sales'].values.reshape(len(df),1)

In [26]:
model_data2 = {
    'N': len(df),
    'max_lag': max_lag, 
    'num_media': num_media,
    'X_media': X_media, 
    'mu_mdip': mu_mdip,
    'num_ctrl': X_ctrl.shape[1],
    'X_ctrl': X_ctrl, 
    'y': df_mmm['sales'].values
}

In [27]:
model_code2 = '''
functions {
  // the adstock transformation with a vector of weights
  real Adstock(vector t, row_vector weights) {
    return dot_product(t, weights) / sum(weights);
  }
}
data {
  // the total number of observations
  int<lower=1> N;
  // the vector of sales
  real y[N];
  // the maximum duration of lag effect, in weeks
  int<lower=1> max_lag;
  // the number of media channels
  int<lower=1> num_media;
  // matrix of media variables
  matrix[N+max_lag-1, num_media] X_media;
  // vector of media variables' mean
  real mu_mdip[num_media];
  // the number of other control variables
  int<lower=1> num_ctrl;
  // a matrix of control variables
  matrix[N, num_ctrl] X_ctrl;
}
parameters {
  // residual variance
  real<lower=0> noise_var;
  // the intercept
  real tau;
  // the coefficients for media variables and base sales
  vector<lower=0>[num_media+num_ctrl] beta;
  // the decay and peak parameter for the adstock transformation of
  // each media
  vector<lower=0,upper=1>[num_media] decay;
  vector<lower=0,upper=ceil(max_lag/2)>[num_media] peak;
}
transformed parameters {
  // the cumulative media effect after adstock
  real cum_effect;
  // matrix of media variables after adstock
  matrix[N, num_media] X_media_adstocked;
  // matrix of all predictors
  matrix[N, num_media+num_ctrl] X;
  
  // adstock, mean-center, log1p transformation
  row_vector[max_lag] lag_weights;
  for (nn in 1:N) {
    for (media in 1 : num_media) {
      for (lag in 1 : max_lag) {
        lag_weights[max_lag-lag+1] <- pow(decay[media], (lag - 1 - peak[media]) ^ 2);
      }
     cum_effect <- Adstock(sub_col(X_media, nn, media, max_lag), lag_weights);
     X_media_adstocked[nn, media] <- log1p(cum_effect/mu_mdip[media]);
    }
  X <- append_col(X_media_adstocked, X_ctrl);
  } 
}
model {
  decay ~ beta(3,3);
  peak ~ uniform(0, ceil(max_lag/2));
  tau ~ normal(0, 5);
  for (i in 1 : num_media+num_ctrl) {
    beta[i] ~ normal(0, 1);
  }
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  y ~ normal(tau + X * beta, sqrt(noise_var));
}
'''

In [28]:
sm2 = pystan.StanModel(model_code=model_code2, verbose=True)
fit2 = sm2.sampling(data=model_data2, iter=10, chains=3)
fit2_result = fit2.extract()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c6677ffefdee0513f144508ee1783d0c NOW.
INFO:pystan:OS: linux, Python: 3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)], Cython 0.28.5


Compiling /tmp/pystan_e_o4ue5b/stanfit4anon_model_c6677ffefdee0513f144508ee1783d0c_6276522429672092095.pyx because it changed.
[1/1] Cythonizing /tmp/pystan_e_o4ue5b/stanfit4anon_model_c6677ffefdee0513f144508ee1783d0c_6276522429672092095.pyx
building 'stanfit4anon_model_c6677ffefdee0513f144508ee1783d0c_6276522429672092095' extension
creating /tmp/pystan_e_o4ue5b/tmp
creating /tmp/pystan_e_o4ue5b/tmp/pystan_e_o4ue5b
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -m64 -fPIC -m64 -fPIC -fPIC -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -I/tmp/pystan_e_o4ue5b -I/opt/conda/lib/python3.6/site-packages/pystan -I/opt/conda/lib/python3.6/site-packages/pystan/stan/src -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3 -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0 -I/opt/conda/lib/python3.6/site-packages/pys

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


# Создание модели semi-log reg + semi-log reg

## Control model

### Создание признаков

Применим к целевому признаку логарифмическую трансформацию

In [29]:
df_ctrl, sc_ctrl = mean_log1p_trandform(df, sales)

Применим к остальным признакам линейное преобразование

In [30]:
df_ctrl2,_ = mean_center_trandform(df,macro_cols + dis_cols + stores_cols)

Соединим две таблицы

In [31]:
df_ctrl = df_ctrl.join(df_ctrl2)
#df_ctrl = df_ctrl.merge(df_ctrl2,how = 'left',on = df_ctrl2.index)

Посмотрим на таблицу

In [32]:
df_ctrl.head(5)

,sales,macro_ics_all,macro_gas_dpg,discount_edw,discount_pdm,stores_count
0,0.509139,0.878350,1.396277,0.0,1.088653,1.087402
1,0.545800,0.878350,1.391228,0.0,1.067596,1.087619
2,0.498112,0.878350,1.378411,0.0,1.020033,1.088486
3,0.490081,0.878350,1.371808,0.0,1.055949,1.088486
4,0.584522,0.900708,1.373362,0.0,1.064874,1.088486


Соединим две таблицы

imp_cols=[col for col in df.columns if 'impressions_' in col]
costs_cols=[col for col in df.columns if 'costs_' in col]
macro_cols = [col for col in df.columns if 'macro_' in col]
dis_cols = [col for col in df.columns if 'discount_' in col]
stores_cols = [col for col in df.columns if 'stores_count' in col]
sales = [col for col in df.columns if 'sales' in col]
holiday = [col for col in df.columns if 'holiday_' in col]
base_vars = macro_cols+dis_cols+stores_cols+holiday+seas_cols

In [33]:
#df_ctrl = df_ctrl.join(df[holiday+seas_cols])
df_ctrl = df_ctrl.merge(df[holiday+seas_cols],how = 'left',on = df.index)

Посмотрим на таблицу

In [34]:
df_ctrl.head(5)

,key_0,sales,macro_ics_all,macro_gas_dpg,discount_edw,discount_pdm,stores_count,holiday_Christmas Day/Christmas Eve/Day after Christmas,holiday_Christmas Day/Day after Christmas,holiday_Christmas Day/Day after Christmas/NYE,...,seas_prd_12,seas_week_40,seas_week_41,seas_week_42,seas_week_43,seas_week_44,seas_week_45,seas_week_46,seas_week_47,seas_week_48
0,0,0.509139,0.878350,1.396277,0.0,1.088653,1.087402,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.545800,0.878350,1.391228,0.0,1.067596,1.087619,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.498112,0.878350,1.378411,0.0,1.020033,1.088486,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.490081,0.878350,1.371808,0.0,1.055949,1.088486,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.584522,0.900708,1.373362,0.0,1.064874,1.088486,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
pos_vars = [col for col in base_vars if col not in seas_cols]
X1 = df_ctrl[pos_vars].values

In [36]:
pn_vars = seas_cols
X2 = df_ctrl[pn_vars].values

### Обучение модели

In [37]:
ctrl_data = {
    'N': len(df_ctrl),
    'K1': len(pos_vars), 
    'K2': len(pn_vars), 
    'X1': X1,
    'X2': X2, 
    'y': df_ctrl['sales'].values,
    'max_intercept': min(df_ctrl['sales'])
}

In [38]:
ctrl_code1 = '''
data {
  int N; // number of observations
  int K1; // number of positive predictors
  int K2; // number of positive/negative predictors
  real max_intercept; // restrict the intercept to be less than the minimum y
  matrix[N, K1] X1;
  matrix[N, K2] X2;
  vector[N] y; 
}

parameters {
  vector<lower=0>[K1] beta1; // regression coefficients for X1 (positive)
  vector[K2] beta2; // regression coefficients for X2
  real<lower=0, upper=max_intercept> alpha; // intercept
  real<lower=0> noise_var; // residual variance
}

model {
  // Define the priors
  beta1 ~ normal(0, 1); 
  beta2 ~ normal(0, 1); 
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  // The likelihood
  y ~ normal(X1*beta1 + X2*beta2 + alpha, sqrt(noise_var));
}
'''

In [39]:
sm1 = pystan.StanModel(model_code=ctrl_code1, verbose=True)
fit1 = sm1.sampling(data=ctrl_data, iter=2000, chains=4)
fit1_result = fit1.extract()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_548939bc33801f8115bc26206558c913 NOW.
INFO:pystan:OS: linux, Python: 3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)], Cython 0.28.5


Compiling /tmp/pystan_ofciu74o/stanfit4anon_model_548939bc33801f8115bc26206558c913_3007344672026232584.pyx because it changed.
[1/1] Cythonizing /tmp/pystan_ofciu74o/stanfit4anon_model_548939bc33801f8115bc26206558c913_3007344672026232584.pyx
building 'stanfit4anon_model_548939bc33801f8115bc26206558c913_3007344672026232584' extension
creating /tmp/pystan_ofciu74o/tmp
creating /tmp/pystan_ofciu74o/tmp/pystan_ofciu74o
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -m64 -fPIC -m64 -fPIC -fPIC -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -I/tmp/pystan_ofciu74o -I/opt/conda/lib/python3.6/site-packages/pystan -I/opt/conda/lib/python3.6/site-packages/pystan/stan/src -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3 -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0 -I/opt/conda/lib/python3.6/site-packages/pys

### Тестирование модели

In [40]:
base_sales_model = extract_ctrl_model(fit1_result, pos_vars=pos_vars, pn_vars=pn_vars)
base_sales = ctrl_model_predict(base_sales_model, df_ctrl)
df['base_sales'] = base_sales*sc_ctrl['sales']
# evaluate control model
print('mape: ', mean_absolute_percentage_error(df['sales'], df['base_sales']))

mape:  28.4878419445


Ошибка на логарифмической модели самую малость меньше =)

## Marketing Mix Model

In [41]:
df_mmm, sc_mmm = mean_log1p_trandform(df, ['sales', 'base_sales'])

mu_mdip = df[imp_cols].apply(np.mean, axis=0).values
max_lag = 8
num_media = len(imp_cols)
# padding zero * (max_lag-1) rows
X_media = np.concatenate((np.zeros((max_lag-1, num_media)), df[imp_cols].values), axis=0)
X_ctrl = df_mmm['base_sales'].values.reshape(len(df),1)

In [42]:
model_data2 = {
    'N': len(df),
    'max_lag': max_lag, 
    'num_media': num_media,
    'X_media': X_media, 
    'mu_mdip': mu_mdip,
    'num_ctrl': X_ctrl.shape[1],
    'X_ctrl': X_ctrl, 
    'y': df_mmm['sales'].values
}

In [43]:
model_code2 = '''
functions {
  // the adstock transformation with a vector of weights
  real Adstock(vector t, row_vector weights) {
    return dot_product(t, weights) / sum(weights);
  }
}
data {
  // the total number of observations
  int<lower=1> N;
  // the vector of sales
  real y[N];
  // the maximum duration of lag effect, in weeks
  int<lower=1> max_lag;
  // the number of media channels
  int<lower=1> num_media;
  // matrix of media variables
  matrix[N+max_lag-1, num_media] X_media;
  // vector of media variables' mean
  real mu_mdip[num_media];
  // the number of other control variables
  int<lower=1> num_ctrl;
  // a matrix of control variables
  matrix[N, num_ctrl] X_ctrl;
}
parameters {
  // residual variance
  real<lower=0> noise_var;
  // the intercept
  real tau;
  // the coefficients for media variables and base sales
  vector<lower=0>[num_media+num_ctrl] beta;
  // the decay and peak parameter for the adstock transformation of
  // each media
  vector<lower=0,upper=1>[num_media] decay;
  vector<lower=0,upper=ceil(max_lag/2)>[num_media] peak;
}
transformed parameters {
  // the cumulative media effect after adstock
  real cum_effect;
  // matrix of media variables after adstock
  matrix[N, num_media] X_media_adstocked;
  // matrix of all predictors
  matrix[N, num_media+num_ctrl] X;
  
  // adstock, mean-center, log1p transformation
  row_vector[max_lag] lag_weights;
  for (nn in 1:N) {
    for (media in 1 : num_media) {
      for (lag in 1 : max_lag) {
        lag_weights[max_lag-lag+1] <- pow(decay[media], (lag - 1 - peak[media]) ^ 2);
      }
     cum_effect <- Adstock(sub_col(X_media, nn, media, max_lag), lag_weights);
     X_media_adstocked[nn, media] <- log1p(cum_effect/mu_mdip[media]);
    }
  X <- append_col(X_media_adstocked, X_ctrl);
  } 
}
model {
  decay ~ beta(3,3);
  peak ~ uniform(0, ceil(max_lag/2));
  tau ~ normal(0, 5);
  for (i in 1 : num_media+num_ctrl) {
    beta[i] ~ normal(0, 1);
  }
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  y ~ normal(tau + X * beta, sqrt(noise_var));
}
'''

In [44]:
sm2 = pystan.StanModel(model_code=model_code2, verbose=True)
fit2 = sm2.sampling(data=model_data2, iter=10, chains=3)
fit2_result = fit2.extract()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c6677ffefdee0513f144508ee1783d0c NOW.
INFO:pystan:OS: linux, Python: 3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)], Cython 0.28.5


Compiling /tmp/pystan_yj4bdhv1/stanfit4anon_model_c6677ffefdee0513f144508ee1783d0c_6786068553463927287.pyx because it changed.
[1/1] Cythonizing /tmp/pystan_yj4bdhv1/stanfit4anon_model_c6677ffefdee0513f144508ee1783d0c_6786068553463927287.pyx
building 'stanfit4anon_model_c6677ffefdee0513f144508ee1783d0c_6786068553463927287' extension
creating /tmp/pystan_yj4bdhv1/tmp
creating /tmp/pystan_yj4bdhv1/tmp/pystan_yj4bdhv1
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -m64 -fPIC -m64 -fPIC -fPIC -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -I/tmp/pystan_yj4bdhv1 -I/opt/conda/lib/python3.6/site-packages/pystan -I/opt/conda/lib/python3.6/site-packages/pystan/stan/src -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3 -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0 -I/opt/conda/lib/python3.6/site-packages/pys

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


# Создание модели log-log reg + log-log reg

В данном случае логарифмическую трансформацию мы применяем не только к зависимой переменной, но и к независимым

## Control model

### Создание признаков

In [45]:
df_ctrl, sc_ctrl = mean_log1p_trandform(df, sales + macro_cols + dis_cols + stores_cols)
df_ctrl = df_ctrl.merge(df[holiday+seas_cols],how = 'left',on = df.index)

In [46]:
pos_vars = [col for col in base_vars if col not in seas_cols]
X1 = df_ctrl[pos_vars].values

In [47]:
pn_vars = seas_cols
X2 = df_ctrl[pn_vars].values

In [48]:
ctrl_data = {
    'N': len(df_ctrl),
    'K1': len(pos_vars), 
    'K2': len(pn_vars), 
    'X1': X1,
    'X2': X2, 
    'y': df_ctrl['sales'].values,
    'max_intercept': min(df_ctrl['sales'])
}

In [49]:
ctrl_code1 = '''
data {
  int N; // number of observations
  int K1; // number of positive predictors
  int K2; // number of positive/negative predictors
  real max_intercept; // restrict the intercept to be less than the minimum y
  matrix[N, K1] X1;
  matrix[N, K2] X2;
  vector[N] y; 
}

parameters {
  vector<lower=0>[K1] beta1; // regression coefficients for X1 (positive)
  vector[K2] beta2; // regression coefficients for X2
  real<lower=0, upper=max_intercept> alpha; // intercept
  real<lower=0> noise_var; // residual variance
}

model {
  // Define the priors
  beta1 ~ normal(0, 1); 
  beta2 ~ normal(0, 1); 
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  // The likelihood
  y ~ normal(X1*beta1 + X2*beta2 + alpha, sqrt(noise_var));
}
'''

In [50]:
sm1 = pystan.StanModel(model_code=ctrl_code1, verbose=True)
fit1 = sm1.sampling(data=ctrl_data, iter=2000, chains=4)
fit1_result = fit1.extract()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_548939bc33801f8115bc26206558c913 NOW.
INFO:pystan:OS: linux, Python: 3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)], Cython 0.28.5


Compiling /tmp/pystan_1tng8j0h/stanfit4anon_model_548939bc33801f8115bc26206558c913_117891232079562589.pyx because it changed.
[1/1] Cythonizing /tmp/pystan_1tng8j0h/stanfit4anon_model_548939bc33801f8115bc26206558c913_117891232079562589.pyx
building 'stanfit4anon_model_548939bc33801f8115bc26206558c913_117891232079562589' extension
creating /tmp/pystan_1tng8j0h/tmp
creating /tmp/pystan_1tng8j0h/tmp/pystan_1tng8j0h
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -m64 -fPIC -m64 -fPIC -fPIC -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -I/tmp/pystan_1tng8j0h -I/opt/conda/lib/python3.6/site-packages/pystan -I/opt/conda/lib/python3.6/site-packages/pystan/stan/src -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3 -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0 -I/opt/conda/lib/python3.6/site-packages/pystan

In [51]:
base_sales_model = extract_ctrl_model(fit1_result, pos_vars=pos_vars, pn_vars=pn_vars)
base_sales = ctrl_model_predict(base_sales_model, df_ctrl)
df['base_sales'] = base_sales*sc_ctrl['sales']
# evaluate control model
print('mape: ', mean_absolute_percentage_error(df['sales'], df['base_sales']))

mape:  28.4969940336


## Marketing Mix Model

Тут вместо стандартного линейного преобразования  к независимым переменным (imp_cols) применим логарифмическое

In [52]:
df_mmm, sc_mmm = mean_log1p_trandform(df, ['sales', 'base_sales'])

mu_mdip = df[imp_cols].apply(np.mean, axis=0).values
max_lag = 8
num_media = len(imp_cols)
# padding zero * (max_lag-1) rows
log_table,_ = mean_log1p_trandform(df[imp_cols],imp_cols)

# X_media - matrix of media variables
# Логарифмируем также независимые переменные
X_media = np.concatenate((np.zeros((max_lag-1, num_media)), log_table), axis=0)

X_ctrl = df_mmm['base_sales'].values.reshape(len(df),1)

In [53]:
model_data2 = {
    'N': len(df),
    'max_lag': max_lag, 
    'num_media': num_media,
    'X_media': X_media, 
    'mu_mdip': mu_mdip,
    'num_ctrl': X_ctrl.shape[1],
    'X_ctrl': X_ctrl, 
    'y': df_mmm['sales'].values
}

In [54]:
model_code2 = '''
functions {
  // the adstock transformation with a vector of weights
  real Adstock(vector t, row_vector weights) {
    return dot_product(t, weights) / sum(weights);
  }
}
data {
  // the total number of observations
  int<lower=1> N;
  // the vector of sales
  real y[N];
  // the maximum duration of lag effect, in weeks
  int<lower=1> max_lag;
  // the number of media channels
  int<lower=1> num_media;
  // matrix of media variables
  matrix[N+max_lag-1, num_media] X_media;
  // vector of media variables' mean
  real mu_mdip[num_media];
  // the number of other control variables
  int<lower=1> num_ctrl;
  // a matrix of control variables
  matrix[N, num_ctrl] X_ctrl;
}
parameters {
  // residual variance
  real<lower=0> noise_var;
  // the intercept
  real tau;
  // the coefficients for media variables and base sales
  vector<lower=0>[num_media+num_ctrl] beta;
  // the decay and peak parameter for the adstock transformation of
  // each media
  vector<lower=0,upper=1>[num_media] decay;
  vector<lower=0,upper=ceil(max_lag/2)>[num_media] peak;
}
transformed parameters {
  // the cumulative media effect after adstock
  real cum_effect;
  // matrix of media variables after adstock
  matrix[N, num_media] X_media_adstocked;
  // matrix of all predictors
  matrix[N, num_media+num_ctrl] X;
  
  // adstock, mean-center, log1p transformation
  row_vector[max_lag] lag_weights;
  for (nn in 1:N) {
    for (media in 1 : num_media) {
      for (lag in 1 : max_lag) {
        lag_weights[max_lag-lag+1] <- pow(decay[media], (lag - 1 - peak[media]) ^ 2);
      }
     cum_effect <- Adstock(sub_col(X_media, nn, media, max_lag), lag_weights);
     X_media_adstocked[nn, media] <- log1p(cum_effect/mu_mdip[media]);
    }
  X <- append_col(X_media_adstocked, X_ctrl);
  } 
}
model {
  decay ~ beta(3,3);
  peak ~ uniform(0, ceil(max_lag/2));
  tau ~ normal(0, 5);
  for (i in 1 : num_media+num_ctrl) {
    beta[i] ~ normal(0, 1);
  }
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  y ~ normal(tau + X * beta, sqrt(noise_var));
}
'''

In [55]:
sm2 = pystan.StanModel(model_code=model_code2, verbose=True)
fit2 = sm2.sampling(data=model_data2, iter=5, chains=3)
fit2_result = fit2.extract()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c6677ffefdee0513f144508ee1783d0c NOW.
INFO:pystan:OS: linux, Python: 3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)], Cython 0.28.5


Compiling /tmp/pystan_v2q6xrw4/stanfit4anon_model_c6677ffefdee0513f144508ee1783d0c_6499017401758746828.pyx because it changed.
[1/1] Cythonizing /tmp/pystan_v2q6xrw4/stanfit4anon_model_c6677ffefdee0513f144508ee1783d0c_6499017401758746828.pyx
building 'stanfit4anon_model_c6677ffefdee0513f144508ee1783d0c_6499017401758746828' extension
creating /tmp/pystan_v2q6xrw4/tmp
creating /tmp/pystan_v2q6xrw4/tmp/pystan_v2q6xrw4
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -m64 -fPIC -m64 -fPIC -fPIC -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -I/tmp/pystan_v2q6xrw4 -I/opt/conda/lib/python3.6/site-packages/pystan -I/opt/conda/lib/python3.6/site-packages/pystan/stan/src -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3 -I/opt/conda/lib/python3.6/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0 -I/opt/conda/lib/python3.6/site-packages/pys

To run all diagnostics call pystan.check_hmc_diagnostics(fit)
